In [102]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import optuna
from sklearn import datasets, metrics
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from statistics import mean

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/icr-identify-age-related-conditions/sample_submission.csv
/kaggle/input/icr-identify-age-related-conditions/greeks.csv
/kaggle/input/icr-identify-age-related-conditions/train.csv
/kaggle/input/icr-identify-age-related-conditions/test.csv


In [103]:
train_df=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/train.csv')
test_df=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')
greeks_df=pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/greeks.csv')
test_Id=test_df["Id"]

In [104]:
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split,GridSearchCV
from sklearn.metrics import roc_auc_score,accuracy_score,confusion_matrix,ConfusionMatrixDisplay,RocCurveDisplay

import time 
from xgboost import XGBClassifier
from sklearn.metrics import log_loss

In [105]:
train_df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [106]:
train_df.dtypes

Id        object
AB       float64
AF       float64
AH       float64
AM       float64
AR       float64
AX       float64
AY       float64
AZ       float64
BC       float64
BD       float64
BN       float64
BP       float64
BQ       float64
BR       float64
BZ       float64
CB       float64
CC       float64
CD       float64
CF       float64
CH       float64
CL       float64
CR       float64
CS       float64
CU       float64
CW       float64
DA       float64
DE       float64
DF       float64
DH       float64
DI       float64
DL       float64
DN       float64
DU       float64
DV       float64
DY       float64
EB       float64
EE       float64
EG       float64
EH       float64
EJ        object
EL       float64
EP       float64
EU       float64
FC       float64
FD       float64
FE       float64
FI       float64
FL       float64
FR       float64
FS       float64
GB       float64
GE       float64
GF       float64
GH       float64
GI       float64
GL       float64
Class      int64
dtype: object

In [107]:
test_df.dtypes

Id      object
AB     float64
AF     float64
AH     float64
AM     float64
AR     float64
AX     float64
AY     float64
AZ     float64
BC     float64
BD     float64
BN     float64
BP     float64
BQ     float64
BR     float64
BZ     float64
CB     float64
CC     float64
CD     float64
CF     float64
CH     float64
CL     float64
CR     float64
CS     float64
CU     float64
CW     float64
DA     float64
DE     float64
DF     float64
DH     float64
DI     float64
DL     float64
DN     float64
DU     float64
DV     float64
DY     float64
EB     float64
EE     float64
EG     float64
EH     float64
EJ      object
EL     float64
EP     float64
EU     float64
FC     float64
FD     float64
FE     float64
FI     float64
FL     float64
FR     float64
FS     float64
GB     float64
GE     float64
GF     float64
GH     float64
GI     float64
GL     float64
dtype: object

In [108]:
num_cols = train_df.select_dtypes(include=["float64"]).columns.tolist()
cat_cols = train_df.select_dtypes(include=["object"]).columns.tolist()
cat_cols.remove("Id")

In [109]:
total_row_train=train_df.shape
total_row_test=test_df.shape
type(total_row_train[0])

int

In [110]:
num=len(train_df[train_df["Class"]==1])
percentage=num/(total_row_train[0])*100
print(f"there are {num} Classes are '1', which is {percentage:.2f}% total training dataset.")

num=len(train_df[train_df["Class"]==0])
percentage=num/(total_row_train[0])*100
print(f"there are {num} Classes are '0', which is {percentage:.2f}% total training dataset.")


there are 108 Classes are '1', which is 17.50% total training dataset.
there are 509 Classes are '0', which is 82.50% total training dataset.


In [111]:
train_df.isnull().sum().loc[train_df.isnull().sum()>0]

BQ    60
CB     2
CC     3
DU     1
EL    60
FC     1
FL     1
FS     2
GL     1
dtype: int64

In [112]:
test_df.isnull().sum().loc[train_df.isnull().sum()>0]

BQ    0
CB    0
CC    0
DU    0
EL    0
FC    0
FL    0
FS    0
GL    0
dtype: int64

In [113]:
train_df.describe()

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
count,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,617.000000,...,616.000000,617.000000,615.000000,617.000000,617.000000,617.000000,617.000000,617.000000,616.000000,617.000000
mean,0.477149,3502.013221,118.624513,38.968552,10.128242,5.545576,0.060320,10.566447,8.053012,5350.388655,...,5.433199,3.533905,0.421501,20.724856,131.714987,14679.595398,31.489716,50.584437,8.530961,0.175041
std,0.468388,2300.322717,127.838950,69.728226,10.518877,2.551696,0.416817,4.350645,65.166943,3021.326641,...,11.496257,50.181948,1.305365,9.991907,144.181524,19352.959387,9.864239,36.266251,10.327010,0.380310
min,0.081187,192.593280,85.200147,3.177522,8.138688,0.699861,0.025578,3.396778,1.229900,1693.624320,...,0.173229,0.497060,0.067730,4.102182,72.611063,13.038894,9.432735,0.897628,0.001129,0.000000
25%,0.252107,2197.345480,85.200147,12.270314,8.138688,4.128294,0.025578,8.129580,1.229900,4155.702870,...,0.173229,0.497060,0.067730,14.036718,72.611063,2798.992584,25.034888,23.011684,0.124392,0.000000
50%,0.354659,3120.318960,85.200147,20.533110,8.138688,5.031912,0.025578,10.461320,1.229900,4997.960730,...,3.028141,1.131000,0.250601,18.771436,72.611063,7838.273610,30.608946,41.007968,0.337827,0.000000
75%,0.559763,4361.637390,113.739540,39.139886,8.138688,6.431634,0.036845,12.969516,5.081244,6035.885700,...,6.238814,1.512060,0.535067,25.608406,127.591671,19035.709240,36.863947,67.931664,21.978000,0.000000
max,6.161666,28688.187660,1910.123198,630.518230,178.943634,38.270880,10.315851,38.971568,1463.693448,53060.599240,...,137.932739,1244.227020,31.365763,135.781294,1497.351958,143790.071200,81.210825,191.194764,21.978000,1.000000


In [114]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score

imputer=SimpleImputer(strategy="mean")
numeric_df=pd.DataFrame(imputer.fit_transform(train_df[num_cols]),columns=num_cols
                )

scaler=MinMaxScaler()
scaled_numeric_df=pd.DataFrame(scaler.fit_transform(numeric_df),columns=num_cols)

encoder=OneHotEncoder(sparse_output=False,handle_unknown="ignore")
encoded_cat_df=pd.DataFrame(encoder.fit_transform(train_df[cat_cols]),columns=encoder.get_feature_names_out(cat_cols))

processed_df=pd.concat([scaled_numeric_df,encoded_cat_df],axis=1)

In [115]:

class Classifier:
    def __init__(self,params):
        self.params=params
        if params["standardize"]=="StandardScaler":
            self.standardizer=StandardScaler()
        elif params["standardize"]=="MinMaxScaler":
            self.standardizer=MinMaxScaler()
        elif params["standardize"]=="Noscaler" :
            self.standardizer=Noscaler()
        
        if params["classifier_name"]=="RandomForest":
            self.classifier=RandomForestClassifier(**params["classifier_params"])
        elif params["classifier_name"]=="XGB":
            self.classifier=XGBClassifier(**params["classifier_params"])
    
    def _fit_and_predict_core(self,x,y=None,fitting=False,proba=False):
        if fitting==True:
            self.standardizer.fit(x)
        self.standardizer.transform(x)
        
        if fitting==True:
            self.classifier.fit(x,y)
        
        if y is None:
            if proba:
                return self.classifier.predict_proba(x)
            else:
                return self.classifier.predict(x)
        return None
    
    def fit(self,x,y):
        self._fit_and_predict_core(x,y,fitting=True)
        return self
    
    def predict(self,x):
        pred_y=self._fit_and_predict_core(x)
        return pred_y
    
    def predict_proba(self,x):
        pred_y=self._fit_and_predict_core(x,proba=True)
        return pred_y
            

In [116]:
class Objective:
    def __init__(self,x,y,label_index):
        self.x=x
        self.y=y
        self.label_index=label_index
        self.best_score=0
        self.best_params=None
    
    def __call__(self,trial):
        x=self.x
        y=self.y
        params=self.generate_params(trial,x)
        
        classifier=Classifier(params)
        
        skf=StratifiedKFold(n_splits=5,random_state=2019,shuffle=True)
        
        acc=[]
        
        for train,test in skf.split(x,y):
            train_y=y.iloc[train].values
            test_y=y.iloc[test].values
            train_x=x.iloc[train].values
            test_x=x.iloc[test].values
            
            classifier.fit(train_x,train_y)
            pred_y=classifier.predict(test_x).reshape(-1).astype(np.int)
            
        
            acc.append(metrics.accuracy_score(test_y,pred_y))
        
        f1_score=mean(acc)
        if self.best_score<f1_score:
            self.best_score-f1_score
            self.best_params=params
        return f1_score
    def generate_params(self,trial,x):
        params={}
        
        params["standardize"]=trial.suggest_categorical("standardize",["StandardScaler","MinMaxScaler"])
        params["classifier_name"]=trial.suggest_categorical("classifier_name",["RandomForest","XGB"])
        classifier_params={}
        if params["classifier_name"]=="RandomForest":
            classifier_params['n_estimators'] = trial.suggest_categorical(
                'rf_n_estimators', [5, 10, 20, 30, 50, 100])
            classifier_params['max_features'] = trial.suggest_categorical(
                'rf_max_features', ["auto", 0.2, 0.4, 0.6, 0.8])
            
            classifier_params['n_jobs'] = -1
        elif params["classifier_name"]=="XGB":
            classifier_params["x_estimators"]=trial.suggest_categorical("rf_x_estimators",[100,300,500,700,1000])
            classifier_params["max_depth"]=trial.suggest_categorical("rf_max_depth",[3,4,5,6,7])
            
        else:
            raise RuntimeError('unspport classifier', params['classifier_name'])
            
        params["classifier_params"]=classifier_params 
        return params

In [117]:
x=np.linspace(0,616,617)
x=list(x)
W=train_df["Class"].values
label,label_index=pd.factorize(W)
label_index

array([1, 0])

In [118]:
import optuna
label_index=train_df.index.tolist
objective = Objective(x=processed_df, y=train_df["Class"], label_index=label_index)
study = optuna.create_study(direction='maximize')
param2=study.optimize(objective, n_trials=5)

[I 2023-05-17 05:43:26,389] A new study created in memory with name: no-name-51b3b384-527c-43f4-a0f3-0afd7771c732
/opt/conda/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred_y

[05:43:27] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:27] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.



/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred_y=classifier.predict(test_x).reshape(-1).astype(np.int)
/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Dep

[05:43:27] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:27] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.



/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred_y=classifier.predict(test_x).reshape(-1).astype(np.int)
/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Dep

...

/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred_y=classifier.predict(test_x).reshape(-1).astype(np.int)
/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Dep

[05:43:28] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:28] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.



/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred_y=classifier.predict(test_x).reshape(-1).astype(np.int)
/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Dep

[05:43:28] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:28] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.



/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  pred_y=classifier.predict(test_x).reshape(-1).astype(np.int)
[I 2023-05-17 05:43:28,692] Trial 3 finished with value: 0.9271308680828744 and parameters: {'standardize': 'StandardScaler', 'classifier_name': 'XGB', 'rf_x_estimators': 300, 'rf_max_depth': 3}. Best is trial 2 with value: 0.9319564647259375.
/tmp/ipykernel_35/1707216753.py:27: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing thi

In [119]:
FOLDS=10
SEED=1004
X=processed_df
y=train_df["Class"]
weights=class_weight.compute_sample_weight("balanced",y)
skf=StratifiedKFold(n_splits=FOLDS,shuffle=True,random_state=SEED)
for fold,(train_idx,val_idx) in enumerate(skf.split(X,y)):
    X_train,y_train=X.iloc[train_idx],y.iloc[train_idx]
    X_valid,y_valid=X.iloc[val_idx],y.iloc[val_idx]
    watchlist=[(X_train,y_train),(X_valid,y_valid)]
    
    model=XGBClassifier(x_estimators=300,max_depth=3)
    model.fit(X_train,y_train,sample_weight=weights[train_idx],eval_set=watchlist,early_stopping_rounds=300,verbose=0)

/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[05:43:46] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:47] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:47] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:47] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:47] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:48] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:48] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:48] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:48] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.

[05:43:49] WARNING: ../src/learner.cc:767: 
Parameters: { "x_estimators" } are not used.



In [120]:
model=RandomForestClassifier(n_estimators=20,random_state=0,max_features=0.2)
skf=StratifiedKFold(n_splits=5,random_state=2019,shuffle=True)
for train,test in skf.split(processed_df,train_df["Class"]):
            train_y=train_df["Class"].iloc[train].values
            test_y=train_df["Class"].iloc[test].values
            train_x=processed_df.iloc[train].values
            test_x=processed_df.iloc[test].values
            
            model.fit(train_x,train_y)

In [121]:
test_df=test_df.drop(["Id"],axis=1)

In [122]:
test_df["EJ_A"]=0
test_df["EJ_B"]=0

In [123]:
test_df["EJ_A"][test_df["EJ"]=="A"]=1
test_df["EJ_B"][test_df["EJ"]=="B"]=1

/tmp/ipykernel_35/441707066.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["EJ_A"][test_df["EJ"]=="A"]=1
/tmp/ipykernel_35/441707066.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df["EJ_B"][test_df["EJ"]=="B"]=1


In [124]:
test_df=test_df.drop(["EJ"],axis=1)

In [99]:
pred_df=pd.DataFrame(model.predict_proba(test_df),columns=["class_0","class_1"])

In [100]:
submission_df = pd.concat([test_Id,pred_df],axis=1)
submission_df.to_csv('submission.csv', index=False)

In [101]:
submission_df

,Id,class_0,class_1
0,00eed32682bb,0.764387,0.235613
1,010ebe33f668,0.764387,0.235613
2,02fa521e1838,0.764387,0.235613
3,040e15f562a2,0.764387,0.235613
4,046e85c7cc7f,0.764387,0.235613
